# Prediction d'éligibilité à un crédit d'après les données historiques de crédit remboursés (target 0) ou non remboursés (target 1)
## Partie Exploration rapide et sauvegarde des données essentielles

In [ ]:
from srfunctions import *
import os

In [ ]:
import pandas as pd

In [ ]:
import plotly
from plotly.offline import iplot

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import seaborn as sns
import random 
init_notebook_mode(connected=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.express as px

import plotly.figure_factory as ff
import plotly.graph_objects as go

pd.set_option('display.max_rows', 200)
import warnings
warnings.filterwarnings("ignore")

## fonction courantes

In [ ]:
# représente les données catégorielle et numérique d'un DF.
def plotolist(alist):
    print(alist.columns,alist.shape)
    floatcols = []
    for i in alist.columns:
        print(alist[i].dtype)
        if ((alist[i].dtype=='float64') | (alist[i].dtype=='int64') | (alist[i].dtype=='uint8')):
            print(i)
            floatcols.append(i)
    plot_data(alist[floatcols]) # pas d'output car pas de float
    categcols = []
    for i in alist.columns:
        if alist[i].dtype==object:
            print(i)
            categcols.append(i)
    plot_data_categ(alist[categcols]) # 
    plt.show()

In [ ]:
# sous fonction pour les données numériques
def plot_data(data):
    fig = plt.figure(figsize=(25, 25))
    for i,feat_idx in enumerate(data.columns):
        ax = fig.add_subplot(14,8, i+1)
        h = ax.hist(data[feat_idx].values, bins=50, color='steelblue', density=True, edgecolor='none')
        ax.set_title(data.columns[i][0:20] , fontsize=14)
        ax.tick_params('x', labelrotation=0)
        plt.title(feat_idx[0:min(len(feat_idx),20)] , y=1.2,rotation = 0,fontsize=14)
        plt.yticks(ha='right')
    fig.tight_layout(pad=3.0)
    plt.show()

In [ ]:
# sous fonction pour les données catégorielles
def plot_data_categ(data):
    fig = plt.figure(figsize=(25, 5))
    for i,feat_idx in enumerate(data.columns):
        ax = fig.add_subplot(4,5,i+1)
        ax=data[feat_idx].value_counts()[0:10].plot(kind='bar')
        ax.set_xticklabels(ax.get_xticklabels(), Fontsize=15, rotation = 65, horizontalalignment='right')
        plt.title(feat_idx[0:min(len(feat_idx),20)] , y=1.2, rotation = 0,fontsize=14)
        plt.yticks(ha='right')
    #fig.tight_layout(pad=8.0)
    plt.subplots_adjust(left=0.1,
                    bottom=0.4, 
                    right=0.9, 
                    top=2.5, 
                    wspace=0.9, 
                    hspace=3)
    plt.show()
categcols = []

In [ ]:
# utilisation d'une fonction de kaggle https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering/notebook
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name, fontsize=30); plt.ylabel('Density', fontsize=30); plt.title('%s Distribution' % var_name, fontsize=30)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)
    
    

# Chargement des données

In [ ]:
rootpath="C:\\Users\\francois\\Documents\\DATASCIENCE\\Projet7\\data\\source\\"

In [ ]:
list = os.listdir(rootpath)  # liste des fichiers données source
print(list)

#### check-up , visualisation des données en table

In [ ]:
# visualisation rapide des fichiers
from IPython.display import display
for i in list:
    data=pd.read_csv(rootpath + i)
    print(i,data.shape)
    display(data.head(4))

#### Chargement des fichiers 1 par 1

In [ ]:
apptest = pd.read_csv(rootpath + 'application_test.csv') # 0
apptrain = pd.read_csv(rootpath + 'application_train.csv') # 1
bureau = pd.read_csv(rootpath + 'bureau.csv') # 2
bureau_balance = pd.read_csv(rootpath + 'bureau_balance.csv') # 3
card_balance = pd.read_csv(rootpath + 'credit_card_balance.csv') # 4
customers = pd.read_csv(rootpath + 'HomeCredit_columns_description.csv') # 5
sellers = pd.read_csv(rootpath + 'installments_payments.csv') # 6
geolocation = pd.read_csv(rootpath + 'POS_CASH_balance.csv') # on laisse tomber ça importe peu de savoir ou le produit a été acheté
previous_app = pd.read_csv(rootpath + 'previous_application.csv') # on laisse tomber ça aussi, pas pertinent pour la question posée a priori
sample_sub = pd.read_csv(rootpath + 'sample_submission.csv') 

#### Visualisation des tailles et format des données de base de la table train

In [ ]:
print(apptest.shape)
print('---')
print(apptrain.shape)
print('******')
print(apptest.info())
print('---')
print(apptrain.info())
print('******')
print(apptest.dtypes)
print('------')
print(apptrain.dtypes)

## Représenation rapide et complète des données numériques et catégorielles

In [ ]:
floatcols=[]
# données numériques
print("DONNEES NUMERIQUES")
for i in apptrain.columns:
    if ((apptrain[i].dtype==float) | (apptrain[i].dtype=='int64') | (apptrain[i].dtype=='uint8')):
        #print(i)
        floatcols.append(i)
plot_data(apptrain[floatcols]) # pas d'output car pas de float

# donnée catégorielles
categcols = []
print("DONNEES CATEGORIELLES")
for i in apptrain.columns:
    if apptrain[i].dtype==object:
        #print(i)
        categcols.append(i)
plot_data_categ(apptrain[categcols]) # 

plt.show()

#### Proportion des valeurs Target sous forme d'histogramme

In [ ]:
fig = px.histogram(apptrain, x="TARGET", barmode='group').update_xaxes(categoryorder = "total descending")
fig.update_layout(bargap=0.2)
fig.update_layout(
    title="TARGET distribution",
    xaxis_title="TARGET values",
    yaxis_title="COUNT",
    legend_title="Legend",
    font=dict(
        family="Courier New, monospace",
        size=36,
        color="RebeccaPurple"
    )
)

fig.show()

#### Représentation des proportions homme-femme-non renseigné

In [ ]:
fig = px.histogram(apptrain, x="TARGET",
             color='CODE_GENDER', barmode='group').update_xaxes(categoryorder = "total descending")
fig.update_layout(
    title="TARGET distribution / GENDER",
    xaxis_title="TARGET values",
    yaxis_title="COUNT",
    legend_title="Legend",
    font=dict(
        family="Courier New, monospace",
        size=36,
        color="RebeccaPurple"
    )
)
fig.show()

#### Représentation sous forme de proportion

In [ ]:
#df = sns.load_dataset("apptrain")
x, y, hue = "TARGET", "proportion", "CODE_GENDER"
hue_order = ["M", "F", "XNA"]
sns.set(font_scale=1.75)
sns.set_theme(style="white", palette="hls")
#sns.color_palette('hls', n_colors = 2)
#sns.color_palette("hls", 8)
(apptrain[x]
 .groupby(apptrain[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue))



On peut observer que parmi chez les hommes la proportion de TARGET=1 et + forte que chez les Femmes.
Que les XNA remboursent toujours.

In [ ]:
# représentation par genre et selon qu'ils détiennent un voiture ou non : cela semble un facteur important
fig = px.histogram(apptrain, x="TARGET",
                   color="CODE_GENDER", barmode='group', 
                   facet_col='FLAG_OWN_CAR').update_xaxes(categoryorder = "total descending")


fig.show()

In [ ]:
#df = sns.load_dataset("apptrain")
x, y, hue = "TARGET", "proportion", "FLAG_OWN_CAR"
hue_order = ["N", "Y"]

(apptrain[x]
 .groupby(apptrain[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue))



Ceux qui possèdent une voiture ont tendance à mieux rembourser

In [ ]:
# representation selon les type d'emprunt
fig = px.histogram(apptrain, x="NAME_CONTRACT_TYPE",
             color="FLAG_OWN_CAR", barmode='group', 
                   facet_col='TARGET').update_xaxes(categoryorder = "total descending")

fig.update_layout(
    title="TARGET distribution / Car Owner",
    xaxis_title="TARGET values",
    yaxis_title="COUNT",
    legend_title="Own Car",
    font=dict(
        family="Courier New, monospace",
        size=26,
        color="RebeccaPurple"
    )
)
fig.show()

In [ ]:
# niveau d'etudes
pie = go.Figure(data=[go.Pie(labels = apptrain['NAME_EDUCATION_TYPE'].value_counts().keys(),
                             values = apptrain['NAME_EDUCATION_TYPE'].value_counts().values)])
iplot(pie)

In [ ]:
# nombre d'enfants
fig = px.histogram(apptrain, x="CNT_CHILDREN", color="TARGET", barmode='group').update_xaxes(categoryorder = "total descending")
fig.show()

In [ ]:
# montant de l'emprunt et target
fig = ff.create_distplot([apptrain[apptrain['TARGET']==0]['AMT_CREDIT'], 
                          apptrain[apptrain['TARGET']==1]['AMT_CREDIT']], 
                         ['target= 0', 'target= 1'],
                         bin_size = [2, 2],
                         show_rug=False,
                         show_hist=False,
                         show_curve=True 
                        )
#fig.update_layout(xaxis = dict(title='AMT_CREDIT'),
#                  yaxis =dict(title='Density'),
#                  barmode='overlay')

fig.update_layout(xaxis = dict(title='AMT_CREDIT'),
                  yaxis =dict(title='Density'),
                  title="AMT_Credit Density",
                  barmode='overlay',          
                  font=dict(
                      family="Courier New, monospace",
                      size=30,
                      color="RebeccaPurple" )
                 )

fig.show()

In [ ]:
# selon le sexe et le montant
fig = px.violin(apptrain, x='CODE_GENDER', y='AMT_CREDIT', 
                color='TARGET', box = True)

fig.update_layout(xaxis = dict(title='AMT_CREDIT / Gender'),
                  yaxis =dict(title='AMT_CREDIT'),
                  title="AMT_Credit vs GENDER vs TARGET",
                  barmode='overlay',          
                  font=dict(
                      family="Courier New, monospace",
                      size=30,
                      color="RebeccaPurple" )
                 )

fig.show()

In [ ]:
# montant du bien acheté et remboursement
fig = ff.create_distplot([apptrain[apptrain['TARGET']==0]['AMT_GOODS_PRICE'].dropna(), 
                          apptrain[apptrain['TARGET']==1]['AMT_GOODS_PRICE'].dropna()], 
                         ['target= 0', 'target= 1'],
                         bin_size = [2, 2],
                         show_rug=False,
                         show_hist=False,
                         show_curve=True 
                        )
fig.update_layout(xaxis = dict(title='AMT_GOODS_PRICE'),
                  yaxis =dict(title='Density'),
                  barmode='overlay')


fig.show()

In [ ]:
# type de bien immo

pie = go.Figure(data=[go.Pie(labels = apptrain['NAME_HOUSING_TYPE'].value_counts().keys(),
                             values = apptrain['NAME_HOUSING_TYPE'].value_counts().values)])
iplot(pie)



In [ ]:
# emploi des clients
fig = px.histogram(apptrain, x="OCCUPATION_TYPE",
                  barmode='group', text_auto='.2s').update_xaxes(categoryorder = "total descending")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
# relation emploi target
fig = px.histogram(apptrain, x="OCCUPATION_TYPE", 
                   color="TARGET", barmode='group').update_xaxes(categoryorder = "total descending")

fig.update_layout(xaxis = dict(title='OCCUPATION TYPE'),
                  yaxis =dict(title='COUNT'),
                  title="OCCUPATION TYPE / TARGET",
                  barmode='relative',          
                  font=dict(
                      family="Courier New, monospace",
                      size=20,
                      color="RebeccaPurple" )
                 )

fig.show()

In [ ]:
# société employant
fig = px.histogram(apptrain, x="ORGANIZATION_TYPE", 
                   color="TARGET", barmode='stack').update_xaxes(categoryorder = "total descending")

fig.update_layout(xaxis = dict(title='ORGANIZATION TYPE'),
                  yaxis =dict(title='COUNT'),
                  title="ORGANIZATION TYPE / TARGET",
                  barmode='stack',          
                  font=dict(
                      family="Courier New, monospace",
                      size=20,
                      color="RebeccaPurple" )
                 )
fig.show()

In [ ]:
# ajout d'une table des historique de candidature au crédit
join_1 = apptrain.join(previous_app, how='left', on='SK_ID_CURR', 
                                rsuffix='_previous')
join_1.head()

In [ ]:
# par type de paiement
fig = px.histogram(join_1, x="NAME_PAYMENT_TYPE",
             color="TARGET", barmode='group').update_xaxes(categoryorder = "total descending")
fig.show()

In [ ]:
# raison d'un crédit refusé
pie = go.Figure(data=[go.Pie(labels = join_1['CODE_REJECT_REASON'].value_counts().keys(),
                             values = join_1['CODE_REJECT_REASON'].value_counts().values)])

iplot(pie)

In [ ]:
customers['Description'].loc[customers['Row']=='CNT_PAYMENT'].values

In [ ]:
# terme du credit precedent
fig = px.histogram(join_1, x="CNT_PAYMENT", color="TARGET", barmode='group')
fig.show()
# il y a un pic à 12 ça veut dire qu'au bout de 12 an(?) il y a beaucoup de chance de reprendre un crédit

In [ ]:
apptrain=join_1.copy() 

### booleanisation ou numérisation

In [ ]:
apptrain["NAME_CONTRACT_TYPE"].loc[apptrain["NAME_CONTRACT_TYPE"]=='Cash loans']=0
apptrain["NAME_CONTRACT_TYPE"].loc[apptrain["NAME_CONTRACT_TYPE"]=='Revolving loans']=1
apptrain['NAME_CONTRACT_TYPE'] = apptrain['NAME_CONTRACT_TYPE'].astype(float)
print(apptrain["NAME_CONTRACT_TYPE"].nunique())

In [ ]:
apptrain["FLAG_OWN_CAR"].loc[apptrain["FLAG_OWN_CAR"]=='N']=0
apptrain["FLAG_OWN_CAR"].loc[apptrain["FLAG_OWN_CAR"]=='Y']=1
apptrain['FLAG_OWN_CAR'] = apptrain['FLAG_OWN_CAR'].astype(float)

In [ ]:
apptrain["FLAG_OWN_REALTY"].loc[apptrain["FLAG_OWN_REALTY"]=='N']=0
apptrain["FLAG_OWN_REALTY"].loc[apptrain["FLAG_OWN_REALTY"]=='Y']=1
apptrain['FLAG_OWN_REALTY'] = apptrain['FLAG_OWN_REALTY'].astype(float)

In [ ]:
apptrain["EMERGENCYSTATE_MODE"].loc[apptrain["EMERGENCYSTATE_MODE"]=='No']=0
apptrain["EMERGENCYSTATE_MODE"].loc[apptrain["EMERGENCYSTATE_MODE"]=='Yes']=1
apptrain['EMERGENCYSTATE_MODE'] = apptrain['EMERGENCYSTATE_MODE'].astype(float)

In [ ]:
# On crée des One Hot Encoder pour les données catégorielles et drope 
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_TYPE_SUITE'], prefix='NAME_TYPE_SUITE',drop_first=True)],axis=1)
apptrain.drop(['NAME_TYPE_SUITE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_INCOME_TYPE'], prefix='NAME_INCOME_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_INCOME_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_EDUCATION_TYPE'], prefix='NAME_EDUCATION_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_EDUCATION_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_FAMILY_STATUS'], 
                                              prefix='NAME_FAMILY_STATUS',drop_first=True)],axis=1)
apptrain.drop(['NAME_FAMILY_STATUS'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_HOUSING_TYPE'], 
                                              prefix='NAME_HOUSING_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_HOUSING_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['OCCUPATION_TYPE'], 
                                              prefix='OCCUPATION_TYPE',drop_first=True)],axis=1)
apptrain.drop(['OCCUPATION_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['ORGANIZATION_TYPE'], 
                                              prefix='ORGANIZATION_TYPE',drop_first=True)],axis=1)
apptrain.drop(['ORGANIZATION_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['FONDKAPREMONT_MODE'], 
                                              prefix='FONDKAPREMONT_MODE',drop_first=True)],axis=1)
apptrain.drop(['FONDKAPREMONT_MODE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['HOUSETYPE_MODE'], 
                                              prefix='HOUSETYPE_MODE',drop_first=True)],axis=1)
apptrain.drop(['HOUSETYPE_MODE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['WALLSMATERIAL_MODE'], 
                                              prefix='WALLSMATERIAL_MODE',drop_first=True)],axis=1)
apptrain.drop(['WALLSMATERIAL_MODE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_CONTRACT_TYPE_previous'], 
                                              prefix='NAME_CONTRACT_TYPE_previous',drop_first=True)],axis=1)
apptrain.drop(['NAME_CONTRACT_TYPE_previous'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['WEEKDAY_APPR_PROCESS_START_previous'], 
                                              prefix='WEEKDAY_APPR_PROCESS_START_previous',drop_first=True)],axis=1)
apptrain.drop(['WEEKDAY_APPR_PROCESS_START_previous'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['FLAG_LAST_APPL_PER_CONTRACT'], 
                                              prefix='FLAG_LAST_APPL_PER_CONTRACT',drop_first=True)],axis=1)
apptrain.drop(['FLAG_LAST_APPL_PER_CONTRACT'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['WEEKDAY_APPR_PROCESS_START'], 
                                              prefix='WEEKDAY_APPR_PROCESS_START',drop_first=True)],axis=1)
apptrain.drop(['WEEKDAY_APPR_PROCESS_START'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['CODE_GENDER'], 
                                              prefix='CODE_GENDER',drop_first=True)],axis=1)
apptrain.drop(['CODE_GENDER'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_CASH_LOAN_PURPOSE'], 
                                              prefix='NAME_CASH_LOAN_PURPOSE',drop_first=True)],axis=1)
apptrain.drop(['NAME_CASH_LOAN_PURPOSE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_CONTRACT_STATUS'], 
                                              prefix='NAME_CONTRACT_STATUS',drop_first=True)],axis=1)
apptrain.drop(['NAME_CONTRACT_STATUS'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_PAYMENT_TYPE'], 
                                              prefix='NAME_PAYMENT_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_PAYMENT_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['CODE_REJECT_REASON'], 
                                              prefix='CODE_REJECT_REASON',drop_first=True)],axis=1)
apptrain.drop(['CODE_REJECT_REASON'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_TYPE_SUITE_previous'], 
                                              prefix='NAME_TYPE_SUITE_previous',drop_first=True)],axis=1)
apptrain.drop(['NAME_TYPE_SUITE_previous'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_CLIENT_TYPE'], 
                                              prefix='NAME_CLIENT_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_CLIENT_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_GOODS_CATEGORY'], 
                                              prefix='NAME_GOODS_CATEGORY',drop_first=True)],axis=1)
apptrain.drop(['NAME_GOODS_CATEGORY'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_PORTFOLIO'], 
                                              prefix='NAME_PORTFOLIO',drop_first=True)],axis=1)
apptrain.drop(['NAME_PORTFOLIO'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_PRODUCT_TYPE'], 
                                              prefix='NAME_PRODUCT_TYPE',drop_first=True)],axis=1)
apptrain.drop(['NAME_PRODUCT_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['CHANNEL_TYPE'], 
                                              prefix='CHANNEL_TYPE',drop_first=True)],axis=1)
apptrain.drop(['CHANNEL_TYPE'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_SELLER_INDUSTRY'], 
                                              prefix='NAME_SELLER_INDUSTRY',drop_first=True)],axis=1)
apptrain.drop(['NAME_SELLER_INDUSTRY'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['NAME_YIELD_GROUP'], 
                                              prefix='NAME_YIELD_GROUP',drop_first=True)],axis=1)
apptrain.drop(['NAME_YIELD_GROUP'],axis=1, inplace=True)
apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['PRODUCT_COMBINATION'], 
                                              prefix='PRODUCT_COMBINATION',drop_first=True)],axis=1)
apptrain.drop(['PRODUCT_COMBINATION'],axis=1, inplace=True)

#apptrain = pd.concat([apptrain,pd.get_dummies(apptrain['EMERGENCYSTATE_MODE'], prefix='EMERGENCYSTATE_MODE')],axis=1)
#apptrain.drop(['EMERGENCYSTATE_MODE'],axis=1, inplace=True)

In [ ]:
# je verifie qu'il n'y a plus de catégories objet dans mon df
for i in apptrain.columns:
    if apptrain[i].dtype==object:
        print(i)
        #categcols.append(i)
#plot_data_categ(apptrain[categcols]) # 

In [ ]:
def print_outlier():
    for indic in apptrain.columns:
        print(apptrain[indic].nunique())
        if apptrain[indic].nunique()>5:
            print(indic)
            #print(apptrain[indic].loc[apptrain[indic]>apptrain[indic].mean()+50*apptrain[indic].std()])
            apptrain.drop(apptrain.loc[apptrain[indic]>apptrain[indic].mean()+50*apptrain[indic].std()].index,axis=0,inplace=True)
print_outlier()
# AU FINAL JE N'APPLIQUE PAS CETTE METHODE NI AUCUNE car on utilise pas de modélisation linéaire a priori donc peu d'impact des outliers
# de plus lors du one hot encoding ça crée des valeurs 1 très minoritaires qui sont détectées comme outlier 

In [ ]:
print("Incom min,max",apptrain["AMT_INCOME_TOTAL"].min(),apptrain["AMT_INCOME_TOTAL"].max())
apptrain["AMT_INCOME_TOTAL"].loc[apptrain["AMT_INCOME_TOTAL"]>apptrain["AMT_INCOME_TOTAL"].mean()+100*apptrain["AMT_INCOME_TOTAL"].std()]

In [ ]:
# exploration 
#plt.scatter(apptrain["TARGET"],apptrain["AMT_INCOME_TOTAL"])
fig = ff.create_distplot([np.log(apptrain[apptrain['TARGET']==0]['AMT_INCOME_TOTAL'].dropna()), 
                          np.log(apptrain[apptrain['TARGET']==1]['AMT_INCOME_TOTAL'].dropna())], 
                         ['target= 0', 'target= 1'],
                         bin_size = [2, 2],
                         show_rug=False,
                         show_hist=False,
                         show_curve=True 
                        )
fig.update_layout(xaxis = dict(title='AMT_INCOME_TOTAL'),
                  yaxis =dict(title='Density'),
                  barmode='overlay')


fig.show()

In [ ]:
# observation des valeurs features classée par taux de remplissage
apptrain.isna().mean().sort_values(ascending=False).to_frame().reset_index().plot()
apptrain.isna().mean().sort_values(ascending=False)[0:5]

In [ ]:
# on verifie qu'il y a bien unicité de numero de client
print(apptrain.SK_ID_CURR.nunique())
print(apptrain.SK_ID_CURR.shape)

On passe les ID client comme index

In [ ]:
apptrain.set_index('SK_ID_CURR')

In [ ]:
#apptrain.drop('SK_ID_CURR',axis=1,inplace=True) 

In [ ]:
# représente les données catégorielle et numérique d'un DF.
def printdtype(alist):
    #print(alist.columns,alist.shape)
    #floatcols = []
    for i in alist.columns:
        #print(alist[i].dtype)
        if ((alist[i].dtype!='float64')):
            print(i,alist[i].dtype,'nval',alist[i].nunique())
            print(alist[i].unique())
            alist[i]=alist[i].astype(float)
printdtype(apptrain)

In [ ]:
apptrain.isna().mean().sort_values(ascending=False)[0:50]

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(apptrain)
X = apptrain
print(imp_mean.transform(X))

In [ ]:
#y=imp_mean.transform(X)

In [ ]:
#print(y.shape)
#print(apptrain.shape)

## fill the nan values en option

In [ ]:
# check qu'il n'y a plus de nan
apptrain.isna().mean().sort_values(ascending=False)[0:50]

## 2 D plot

In [ ]:
# Downsampling Resemapling manuel
df1 = apptrain[apptrain.TARGET == 1].sample(frac=0.05)
df2 = apptrain[apptrain.TARGET == 0].sample(frac=0.005)
print(df1.shape)
print(df2.shape)

In [ ]:
df=pd.concat([df1,df2])
df=df.iloc[np.random.permutation(len(df))]

In [ ]:
df

In [ ]:
df.isna().mean().sort_values(ascending=False)[0:50]

In [ ]:
from random import Random
list_selfeat=df.columns[df[df.columns[0::]].nunique()>50] # on ne prend les indicateurs qui sont suffisamment continus pas > un nb de valeurs discretes
#list_selfeat=list_selfeat.drop(['SK_ID_CURR'])[random.sample(range(0, list_selfeat.shape[0]-1), 5)] # # selection d'un sous echantillon d'indicateur
list_selfeat

In [ ]:
list_selfeat1=list_selfeat.union(['TARGET'])

In [ ]:
list_selfeat1 #.astype(list()) #.append('Target')
#list_selptarget

In [ ]:
df[df.TARGET ==1].shape

In [ ]:
df.isna().mean().sort_values(ascending=False)[0:50]

In [ ]:
# genere le grpahe de base des correltion
#g=sns.pairplot(df[list_selfeat1],kind="scatter",  x_vars = df[list_selfeat], y_vars=df[list_selfeat], diag_kind="auto",hue = "TARGET", height=4.5, aspect=1.5, plot_kws={"s": 400})

#for i,ax in enumerate(g.axes.flatten()):
#    ax.set_xticklabels(ax.get_xticklabels(), Fontsize=50, rotation = 45)
#    ax.tick_params(axis='y', labelsize=50)#ax.set_yticklabels(ax.get_yticklabels(), Fontsize=30, rotation = 0)
#    ax.set_xlabel('\n'.join(wrap(ax.get_xlabel(),40) ), fontsize=20, rotation = 20, horizontalalignment='right')
#    ax.set_ylabel('\n'.join(wrap(ax.get_ylabel(),40) ), fontsize=20, rotation = 0, horizontalalignment='right')

#g._legend.remove()
#handles = g._legend_data.values()
#labels = g._legend_data.keys()
#g.fig.legend(handles=handles, labels=labels, loc='right', ncol=1, bbox_to_anchor = (1.05,0.75), fontsize=20, markerscale=10)
#plt.show()

In [ ]:
#g = sns.pairplot(df[list_selfeat], diag_kind="kde")

In [ ]:
#df.isna().mean().sort_values(ascending=False)[0:50]

## HEATMAP

In [ ]:
df.dtypes

In [ ]:
sns.color_palette("coolwarm", as_cmap=True)
mask = np.zeros_like(df.corr()) # je mets un index sur ListRed pour ne pas prendre les indices en compte
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(50,50))
#ax = sns.heatmap(corr, annot=True, fmt=".2f", annot_kws={'size':8}, 
#                 mask=mask, center=0, cmap="coolwarm")

heat_map=sns.heatmap(df.corr(),vmin=-1,vmax=1,annot=False,square=True, center=0,cmap="coolwarm",mask=mask)
heat_map.set_xticklabels(heat_map.get_xticklabels(), fontsize=20, rotation=90, horizontalalignment='right')
heat_map.set_yticklabels(heat_map.get_yticklabels(), fontsize=20, rotation=0, horizontalalignment='right')
plt.show()
# LA eventuellement on pourrait faire une heatmap organisée un peu mieux

In [ ]:
df.corr().shape

In [ ]:
# recherche des features les plus corrélées à la valeur de la target
df.corr().TARGET.sort_values(ascending=True)[0:5]

#### On garde pour la représentation que le 5 premiers features les plus corrélés à la Target

In [ ]:
# représentation des densités de distributions des varaibles les plus corrélées + selon la valeur de la target
for i in df.corr().TARGET.sort_values(ascending=False)[1:5].index:
    kde_target(i, apptrain)
    

In [ ]:
# représentation des densités de distributions des varaibles les plus corrélées - selon la valeur de la target
for i in df.corr().TARGET.sort_values(ascending=True)[1:5].index:
    kde_target(i, apptrain)
    

#### Constitution/selection d'une liste d'indicateurs dont la le facteur de correlation en valeur absolue dépasse 0.1

In [ ]:
list_indic=df.corr()['TARGET'][df.corr()['TARGET'].abs()>0.15]
list_indic.index

In [ ]:
# Heatmap of correlations
best_data_corrs = df[list_indic.index].corr()
best_data_corrs

In [ ]:
plt.figure(figsize = (16, 12))
res=sns.heatmap(best_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.5, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap',fontsize=28);
res.set_xticklabels(res.get_xmajorticklabels(), rotation=45, horizontalalignment='right',fontsize = 18)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 18)


In [ ]:
selecdf=df[best_data_corrs.columns].drop(columns=df[best_data_corrs.columns[best_data_corrs.columns!='TARGET']].columns[df[best_data_corrs.columns[best_data_corrs.columns!='TARGET']].nunique()<10],axis=0)
#seldf=df[list_indic.index].drop(columns = ['REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY']).copy()

In [ ]:
selecdf#best_data_corrs.columns[best_data_corrs.columns!='TARGET']#selecdf.columns.shape

In [ ]:
# Copy the data for plotting
#plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
#plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Drop na values and limit to first 100000 rows
#plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)
matplotlib.rcParams["axes.labelsize"] = 20
# Create the pairgrid object
grid = sns.PairGrid(data = selecdf, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in selecdf.columns if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);
grid.add_legend()
plt.suptitle('Best Features Pairs Plot', size = 32, y = 1.05);


In [ ]:
list_forTarget=list_indic.index #.shape

### Correl plot

In [ ]:
list_forTarget_withoutTartget=df[list_forTarget].columns.drop(['TARGET'])
list_forTarget_withoutTartget

In [ ]:
# genere le grpahe de base des correltion
g=sns.pairplot(df[list_forTarget],kind="scatter",  x_vars = df[list_forTarget].columns.drop(['TARGET']), y_vars=df[list_forTarget].columns.drop(['TARGET']), diag_kind="auto",hue = "TARGET", height=4.5, aspect=1.5, plot_kws={"s": 400})

# gere les axes et labels
for i,ax in enumerate(g.axes.flatten()):
    ax.set_xticklabels(ax.get_xticklabels(), Fontsize=50, rotation = 45)
    ax.tick_params(axis='y', labelsize=50)#ax.set_yticklabels(ax.get_yticklabels(), Fontsize=30, rotation = 0)
    ax.set_xlabel('\n'.join(wrap(ax.get_xlabel(),40) ), fontsize=20, rotation = 20, horizontalalignment='right')
    ax.set_ylabel('\n'.join(wrap(ax.get_ylabel(),40) ), fontsize=20, rotation = 0, horizontalalignment='right')


# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# preparation des données pour une pca
dfpca=df.copy()

In [ ]:
# on garde les données un minimum continues plus de 20 valeurs differentes
dfpca=df.drop(columns=df.columns[df.nunique()<20],axis=0) 

In [ ]:
imp_mean.fit(dfpca)
#SimpleImputer()
X = dfpca #apptrain
#print(imp_mean.transform(X))

In [ ]:
X=imp_mean.transform(X)
X.shape

In [ ]:
from sklearn import preprocessing

# Centrage et Réduction
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

In [ ]:
pca=PCA(n_components=X_scaled.shape[1]-2)

In [ ]:
x_pca = pca.fit_transform(X_scaled)

In [ ]:
ratio=pca.explained_variance_ratio_.cumsum()
plt.plot(range(len(ratio)),ratio)

In [ ]:
pca.explained_variance_ratio_.cumsum().max()

In [ ]:
pca=PCA(n_components=30)  # on prend les features qui suffisent à expliquer 80% dela variance
x_pca = pca.fit_transform(X_scaled) # 
x_pca.shape

In [ ]:
# eventuellement iterative imputer
# create subtrain subtest
# dispplot 2 D hue categories
df.shape

In [ ]:
# creation d'un df avec pca
df3=pd.DataFrame(x_pca)

In [ ]:
#df4=pd.merge(df, df3, left_index=True, right_index=True)
#df4

In [ ]:
#df['DAYS_BIRTH'].dtype

### on sauve certains trucs et on en droppe d'autres

## FEATURE ENGINEERING

In [ ]:
# on peut créer des variables supplémentaires

In [ ]:
#app_train_domain = df.copy()
apptrain['CREDIT_INCOME_PERCENT'] = apptrain['AMT_CREDIT'] / apptrain['AMT_INCOME_TOTAL']
apptrain['ANNUITY_INCOME_PERCENT'] = apptrain['AMT_ANNUITY'] / apptrain['AMT_INCOME_TOTAL']
apptrain['CREDIT_TERM'] = apptrain['AMT_ANNUITY'] / apptrain['AMT_CREDIT']
apptrain['DAYS_EMPLOYED_PERCENT'] = apptrain['DAYS_EMPLOYED'] / apptrain['DAYS_BIRTH']

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(apptrain.loc[apptrain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(apptrain.loc[apptrain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

### on sauve les données nettoyées (indicateurs selectionnés et fillnan..) dans un fichier intermédiaire

On cherche les features qui sont un minimum corrélés avec la Target pour créer un df réduit

In [ ]:
list_indic=apptrain.corr()['TARGET'][apptrain.corr()['TARGET'].abs()>0.05]
list_indic

In [ ]:
print(apptrain.loc[apptrain['TARGET']==1].shape)
print(apptrain.loc[apptrain['TARGET']==0].shape)

#### Sauvegarde d'un df réduit

In [ ]:
apptrain_0=apptrain[list_indic.index].copy()
apptrain_0.to_csv("data/cleaned/ApptrainSaved0.csv",index=False)

#### Sauvegarde de tout le df étendu

In [ ]:
apptrain.to_csv("data/cleaned/ApptrainSaved1.csv",index=False)
